# S12_T02_Web_Scraping



### Ex1: Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium

In [1]:
#Importem llibreries necessàries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup
import re
from IPython.display import Image

import warnings

warnings.filterwarnings('ignore')

In [2]:
#variable URL a la q assignem el link de la pàgina web 
URL = "https://www.bolsamadrid.es"
page = requests.get(URL)

In [3]:
bs = BeautifulSoup(page.content, 'html.parser')

In [4]:
#ens interessa consultar el llistat d'empreses q cotitzen en borsa, així farem un dataframe del llistat
#generem una nova URL directa al llistat q volem obtenir
URL_empreses = "https://www.bolsamadrid.es/esp/aspx/Empresas/Empresas.aspx"
html_empreses = requests.get(URL_empreses)

In [5]:
bs_empreses = BeautifulSoup(html_empreses.content, "html.parser")
bs_empreses


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head data-analytics-id="UA-35966870-2" data-app-path="/" data-bolsa="BMadrid" data-hora-act="Wed, 22 Jun 2022 13:14:35 GMT" data-idioma="esp"><meta content="IE=11" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="Copyright © BME 2022" id="ctl00_copyright" name="copyright"/><title>
	Bolsa de Madrid - Lista de Empresas Cotizadas
</title><link href="/esp/aspx/RSS/RSS.ashx?feed=Todo" id="ctl00_RSSLink1" rel="alternate" title="Bolsa de Madrid: Todos los contenidos agregados" type="application/rss+xml"/><link href="/esp/aspx/RSS/RSS.ashx?feed=NotasPrensa" id="ctl00_RSSLink2" rel="alternate" title="Bolsa de Madrid: Notas de Prensa" type="application/rss+xml"/><link href="/esp/aspx/RSS/RSS.ashx?feed=Regulacion" id="ctl00_RSSLink3" rel="alternate" title="Bolsa de Ma

In [6]:
results_tabla = bs_empreses.find(id="ctl00_Contenido_tblEmisoras")

#busquem els headers de la taula "th" (table header)
results_tabla.find_all("th")

[<th scope="col">Nombre</th>,
 <th scope="col">Sector - Subsector</th>,
 <th scope="col">Mercado</th>,
 <th class="Ult" scope="col">Índices</th>]

In [7]:
print(results_tabla.prettify())

<table cellpadding="3" cellspacing="0" class="TblPort" id="ctl00_Contenido_tblEmisoras" width="100%">
 <tr align="center">
  <th scope="col">
   Nombre
  </th>
  <th scope="col">
   Sector - Subsector
  </th>
  <th scope="col">
   Mercado
  </th>
  <th class="Ult" scope="col">
   Índices
  </th>
 </tr>
 <tr>
  <td>
   <a href="/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0105200002">
    ABENGOA, S.A.
   </a>
  </td>
  <td>
   Mat.Basicos, Industria y Construcción - Ingeniería y Otros
  </td>
  <td>
   Mercado Continuo
  </td>
  <td class="Ult">
  </td>
 </tr>
 <tr>
  <td>
   <a href="/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0125220311">
    ACCIONA,S.A.
   </a>
  </td>
  <td>
   Mat.Basicos, Industria y Construcción - Construcción
  </td>
  <td>
   Mercado Continuo
  </td>
  <td class="Ult">
   IBEX 35®
  </td>
 </tr>
 <tr>
  <td>
   <a href="/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0132105018">
    ACERINOX, S.A.
   </a>
  </td>
  <td>
   Mat.Basicos, Industria y Construcción - Minera

In [8]:
#obtenim el dataframe de la taula que ens interesa i que estem fent scraping
columns = []

for element in results_tabla.find_all("th"):    
    columns.append(element.get_text())
    
print(columns)

['Nombre', 'Sector - Subsector', 'Mercado', 'Índices']


In [9]:
cells = []
for element in results_tabla.find_all("td"):
    cells.append(element.text)
    
print(cells)

['ABENGOA, S.A.', 'Mat.Basicos, Industria y Construcción - Ingeniería y Otros', 'Mercado Continuo', '\xa0', 'ACCIONA,S.A.', 'Mat.Basicos, Industria y Construcción - Construcción', 'Mercado Continuo', 'IBEX 35®', 'ACERINOX, S.A.', 'Mat.Basicos, Industria y Construcción - Mineral, Metales y Transformación', 'Mercado Continuo', 'IBEX 35®, IBEX TOP Dividendo®', 'ACS,ACTIVIDADES DE CONST.Y SERVICIOS S.A', 'Mat.Basicos, Industria y Construcción - Construcción', 'Mercado Continuo', 'IBEX 35®, IBEX TOP Dividendo®', 'ADOLFO DOMINGUEZ, S.A.', 'Bienes de Consumo - Textil, Vestido y Calzado', 'Mercado Continuo', '\xa0', 'AEDAS HOMES, S.A.', 'Servicios Inmobiliarios - Inmobiliarias y Otros', 'Mercado Continuo', '\xa0', 'AENA, S.M.E., S.A.', 'Servicios de Consumo - Transporte y Distribución', 'Mercado Continuo', 'IBEX 35®', 'AIRBUS SE', 'Mat.Basicos, Industria y Construcción - Aerospacial', 'Mercado Continuo', '\xa0', 'AIRTIFICIAL INTELLIGENCE STRUCTURES S.A.', 'Mat.Basicos, Industria y Construcción

In [10]:
#creem diccionari i guardem les dades saltant de 4 en 4, ja que així obtenim les dades de totes les columnes
dades = {}

for i in range(4):
    dades[columns[i]] = cells[i::4]

In [11]:
#fem servir pandas per crear el dataframe a partir del diccionari q hem creat per extreure totes les dades
df_empreses = pd.DataFrame(dades)

In [12]:
#ja hem obtingut el llistat de empreses que voliem!
df_empreses

,Nombre,Sector - Subsector,Mercado,Índices
0,"ABENGOA, S.A.","Mat.Basicos, Industria y Construcción - Ingeni...",Mercado Continuo,
1,"ACCIONA,S.A.","Mat.Basicos, Industria y Construcción - Constr...",Mercado Continuo,IBEX 35®
2,"ACERINOX, S.A.","Mat.Basicos, Industria y Construcción - Minera...",Mercado Continuo,"IBEX 35®, IBEX TOP Dividendo®"
3,"ACS,ACTIVIDADES DE CONST.Y SERVICIOS S.A","Mat.Basicos, Industria y Construcción - Constr...",Mercado Continuo,"IBEX 35®, IBEX TOP Dividendo®"
4,"ADOLFO DOMINGUEZ, S.A.","Bienes de Consumo - Textil, Vestido y Calzado",Mercado Continuo,
5,"AEDAS HOMES, S.A.",Servicios Inmobiliarios - Inmobiliarias y Otros,Mercado Continuo,
6,"AENA, S.M.E., S.A.",Servicios de Consumo - Transporte y Distribución,Mercado Continuo,IBEX 35®
7,AIRBUS SE,"Mat.Basicos, Industria y Construcción - Aerosp...",Mercado Continuo,
8,AIRTIFICIAL INTELLIGENCE STRUCTURES S.A.,"Mat.Basicos, Industria y Construcción - Ingeni...",Mercado Continuo,
9,"ALANTRA PARTNERS, S.A.",Servicios Financieros - Cartera y Holding,Mercado Continuo,


In [13]:
#Selenium: obrim la url i fem el mateix que amb BeautifulSoup

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches",["ignore-certificate-errors"])
options.add_argument('--disable-gpu')
options.add_argument('--headless')
chrome_driver_path = "C:\Python27\Scripts\chromedriver.exe"

In [14]:
options.headless = True
assert options.headless  # Operating in headless mode
url = "https://www.bolsamadrid.es"
browser = webdriver.Chrome()
browser.get(url)

In [15]:
#tornem a buscar ja en la URL del llistat d'empreses
URL_empreses = "https://www.bolsamadrid.es/esp/aspx/Empresas/Empresas.aspx"
browser.get(URL_empreses)

In [16]:
#ara q ja estem a la pàgina que volem, busquem els valors de la taula
results_tabla_sel = browser.find_element_by_id("ctl00_Contenido_tblEmisoras")

In [17]:
columns_sel=[]

for element in results_tabla_sel.find_elements_by_tag_name("th"):    
    columns_sel.append(element.text)
    
print(columns_sel)    

['Nombre', 'Sector - Subsector', 'Mercado', 'Índices']


In [18]:
cells_sel = []
for element in results_tabla_sel.find_elements_by_tag_name("td"):
    cells_sel.append(element.text)
    
print(cells_sel)

['ABENGOA, S.A.', 'Mat.Basicos, Industria y Construcción - Ingeniería y Otros', 'Mercado Continuo', ' ', 'ACCIONA,S.A.', 'Mat.Basicos, Industria y Construcción - Construcción', 'Mercado Continuo', 'IBEX 35®', 'ACERINOX, S.A.', 'Mat.Basicos, Industria y Construcción - Mineral, Metales y Transformación', 'Mercado Continuo', 'IBEX 35®, IBEX TOP Dividendo®', 'ACS,ACTIVIDADES DE CONST.Y SERVICIOS S.A', 'Mat.Basicos, Industria y Construcción - Construcción', 'Mercado Continuo', 'IBEX 35®, IBEX TOP Dividendo®', 'ADOLFO DOMINGUEZ, S.A.', 'Bienes de Consumo - Textil, Vestido y Calzado', 'Mercado Continuo', ' ', 'AEDAS HOMES, S.A.', 'Servicios Inmobiliarios - Inmobiliarias y Otros', 'Mercado Continuo', ' ', 'AENA, S.M.E., S.A.', 'Servicios de Consumo - Transporte y Distribución', 'Mercado Continuo', 'IBEX 35®', 'AIRBUS SE', 'Mat.Basicos, Industria y Construcción - Aerospacial', 'Mercado Continuo', ' ', 'AIRTIFICIAL INTELLIGENCE STRUCTURES S.A.', 'Mat.Basicos, Industria y Construcción - Ingenierí

In [19]:
#creem diccionari i guardem les dades saltant de 4 en 4, ja que així obtenim les dades de totes les columnes
dades_sel = {}

for i in range(4):
    dades[columns_sel[i]] = cells_sel[i::4]

In [20]:
#fem servir pandas per crear el dataframe a partir del diccionari q hem creat per extreure totes les dades
df_empreses_sel = pd.DataFrame(dades_sel)

In [21]:
browser.close()

### Ex2: Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle

### Ex3: Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy